## 바텀 라인의 영향력 분석

In [1]:
import pandas as pd
import requests
from urllib.parse import quote as quotekr # 한글 아이디를 인코딩 parse.quote
import time

In [2]:
api_key = 'RGAPI-547e5336-c2d4-4db0-8298-1bc1b0f5d74a'
request_header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }

#### 티어별 유저 5명 선별
* op.gg에서 무작위 추출
* Most top 3 챔피언이 모두 원거리 딜러
* 랭크 선호 포지션이 바텀 비율이 월등히 높은 사람
* 솔로랭크 50판 이상

In [ ]:
grand_master_user = ['줄라이77', '바텀개', 'always crying', '침대위섹시카이팅', '미드KING']
diamond_user = ['세혀닝', '강한남자한만윤', 'FEARN0T', '잘해봐요괜찮아요', '쿠로미짱이현']
# emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '주희보고싶은민교', 'Asnm']
emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '김잼니', 'Asnm']
platinum_user = ['협곡의 맙소사', '엄딴식', '지구뚝배기', '08년생 큐티정민', 'Gucci Vs']
gold_user = ['루픠는못말려', '홍구형', '카이팅의 귀재', 'Orangerie', '쥐파티']

grand_master_puuid = []
diamond_puuid = []
emerald_puuid = []
platinum_puuid = []
gold_puuid = []

In [ ]:
def user_puuid(users, puuids):
    for user in users:
        url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
        response = requests.get(url, headers=request_header)
        
        if response.status_code == 200: # Success
            pass

        elif response.status_code == 429: # Rate limit exceeded
            print('Rate limit exceeded error, api cost full')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 3 second wait time')
                    time.sleep(3)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 503: # Service unavailable
            print('Service unavailable error')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 5 second wait time')
                    time.sleep(5)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 403: # Forbidden
            print('Forbidden error, you need api renewal')
            print('break')
            return

        puuids.append(response.json()['puuid'])

In [ ]:
# def user_puuid2(users, puuids):
#     for user in users:
#         url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
#         response = requests.get(url, headers=request_header)

#         if response.status_code == 200:
#             pass
        
#         elif response.status_code == 429:
#             print("sleep", user, ' ', response.headers['Retry-After'])
#             time.sleep(int(response.headers['Retry-After']) + 5)
#             response = requests.get(url, headers=request_header)

#         puuids.append(response.json()['puuid'])

In [ ]:
user_puuid(grand_master_user, grand_master_puuid)
user_puuid(diamond_user, diamond_puuid)
user_puuid(emerald_user, emerald_puuid)
user_puuid(platinum_user, platinum_puuid)
user_puuid(gold_user, gold_puuid)

In [ ]:
grand_master_df = pd.DataFrame({
    'tier': ['grand_master' for i in range(5)],
    'summoner': grand_master_user,
    'puuid': grand_master_puuid
    })

diamond_df = pd.DataFrame({
    'tier': ['diamond' for i in range(5)],
    'summoner': diamond_user,
    'puuid': diamond_puuid
    })

emerald_df = pd.DataFrame({
    'tier': ['emerald' for i in range(5)],
    'summoner': emerald_user,
    'puuid': emerald_puuid
    })

platinum_df = pd.DataFrame({
    'tier': ['platinum' for i in range(5)],
    'summoner': platinum_user,
    'puuid': platinum_puuid
    })

gold_df = pd.DataFrame({
    'tier': ['gold' for i in range(5)],
    'summoner': gold_user,
    'puuid': gold_puuid
    })

In [ ]:
df = pd.concat([grand_master_df, diamond_df, emerald_df, platinum_df, gold_df], ignore_index=True)
display(df.head(2))
display(df.tail(2))

In [6]:
df.to_excel('puuid_data2.xlsx', index=False)

In [3]:
df = pd.read_excel('puuid_data2.xlsx')
display(df.head(2))
display(df.tail(2))


,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


In [4]:
df.loc[10:15]

,tier,summoner,puuid
10,emerald,정의는 승리한다v,wXvN9TJlkwSje56uoPtofaIgEoVCUGD63Hewfjw0GjpHO0...
11,emerald,no return0,GqsVg3QvhFw--4L0ascFso9yMh0eKrz9FdKtCf_leW4UOm...
12,emerald,Cigar3,eS7tRJlBcG3HdFA9LH0zq1s1wXzwMHKRgnQFMWYvwsf0Fk...
13,emerald,주희보고싶은민교,Xu3Qh7PO0LgD2nSaH4aUy_iShItuHwcXW5hw-r0yE41Pcv...
14,emerald,Asnm,ev4ifUpwp6BDTaM8r2UERk6flI2aqfprAfUHho3M8rPPey...
15,platinum,협곡의 맙소사,tehmv7dX7eC5ZTZZjrzVIsTMBCb7J07UVBSQjSYK-5PoUJ...


In [5]:
# 주희보고싶은민교
# 포지션 변경으로 적정 데이터 수집 불가, 소환사 변경
df.loc[13] = ['emerald', '김잼니', 'LiQXFaJ4mVIhWVm511xSCIs6MS86Dt_awK6InfFBVKYgjZ_Tt-Ldzc1z7m2Q5iee8Ooj-17HEqn0qg']
df.loc[10:15]

,tier,summoner,puuid
10,emerald,정의는 승리한다v,wXvN9TJlkwSje56uoPtofaIgEoVCUGD63Hewfjw0GjpHO0...
11,emerald,no return0,GqsVg3QvhFw--4L0ascFso9yMh0eKrz9FdKtCf_leW4UOm...
12,emerald,Cigar3,eS7tRJlBcG3HdFA9LH0zq1s1wXzwMHKRgnQFMWYvwsf0Fk...
13,emerald,김잼니,LiQXFaJ4mVIhWVm511xSCIs6MS86Dt_awK6InfFBVKYgjZ...
14,emerald,Asnm,ev4ifUpwp6BDTaM8r2UERk6flI2aqfprAfUHho3M8rPPey...
15,platinum,협곡의 맙소사,tehmv7dX7eC5ZTZZjrzVIsTMBCb7J07UVBSQjSYK-5PoUJ...


#### 데이터 수집
* '바텀 게임' 이라는 말이 줄기 시작한 13.6패치가 3월 중순
* 1주일에 랭크게임 5판 한다고 가정
* 현재 8월 말이므로 약 20주
* 100게임 이전 인당 10판

In [8]:
def find_partiId(mat_data_resp, puuid):
    return mat_data_resp.json()['metadata']['participants'].index(puuid)

In [13]:
def puuid_to_matchid(puuid):
    match_ids = []
    start = 120
    count = 40

    while len(match_ids) < 10: # 10경기 추출
        matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}'
        match_list_resp = requests.get(matches_url, headers=request_header)

        while match_list_resp.status_code == 429: # Rate limit exceed
            print(retry_count, " sleep match_list", match_list_resp.headers['Retry-After'])
            retry_count += 1
            time.sleep(int(match_list_resp.headers['Retry-After']) + 5)
            match_list_resp = requests.get(matches_url, headers=request_header)

        if match_list_resp.status_code == 200:
            for mat_id in match_list_resp.json():
                if len(match_ids) == 10:
                    break

                mat_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mat_id
                mat_data_resp = requests.get(mat_url, headers=request_header)

                while mat_data_resp.status_code == 429: # Rate limit exceed
                    print(retry_count, " sleep mat_data", mat_data_resp.headers['Retry-After'])
                    retry_count += 1
                    time.sleep(int(mat_data_resp.headers['Retry-After']) + 5)
                    mat_data_resp = requests.get(mat_url, headers=request_header)

                if mat_data_resp.status_code == 200:
                    if mat_data_resp.json()['info']['gameMode'] == 'CLASSIC' \
                        and len(mat_data_resp.json()['info']['teams'][0]['bans']) > 0 \
                        and mat_data_resp.json()['info']['participants'][0]['gameEndedInEarlySurrender'] == False \
                        and mat_data_resp.json()['info']['gameDuration'] > 1500 \
                        and mat_data_resp.json()['info']['participants'][find_partiId(mat_data_resp, puuid)]['teamPosition'] == 'BOTTOM':
                        # 소환사 협곡
                        # 밴이 있음
                        # 재시작 경기 x
                        # 15분 이상
                        # 해당 소환사가 바텀라인

                        match_ids.append(mat_id)

                else:
                    print('\n\terror ', mat_data_resp.status_code)
                    print('\t', puuid)
                    break

            start += count

        else:
            print('\n\terror ', match_list_resp.status_code)
            print('\t', match_ids)
            break

    return match_ids

In [10]:
col = [('match' + str(x)) for x in range(1, 11)]
df_puuid = pd.DataFrame(columns=col)
df_puuid

,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10


In [11]:
display(df.head(2))
display(df.tail(2))

,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


In [ ]:
for puuid in df['puuid']:
    matchid_list = puuid_to_matchid(puuid)
    df_mid = pd.DataFrame([matchid_list[::-1]], columns=col)
    df_puuid = pd.concat([df_puuid, df_mid], ignore_index=True)

In [14]:

match_ids = []
start = 110
count = 20
retry_count = 0

while len(match_ids) < 10: # 10경기 추출
    matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}'
    match_list_resp = requests.get(matches_url, headers=request_header)

    while match_list_resp.status_code == 429: # Rate limit exceed
        print(retry_count, " sleep match_list", match_list_resp.headers['Retry-After'])
        retry_count += 1
        time.sleep(int(match_list_resp.headers['Retry-After']) + 5)
        match_list_resp = requests.get(matches_url, headers=request_header)

    if match_list_resp.status_code == 200:
        for mat_id in match_list_resp.json():
            if len(match_ids) == 10:
                break

            mat_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mat_id
            mat_data_resp = requests.get(mat_url, headers=request_header)

            while mat_data_resp.status_code == 429: # Rate limit exceed
                print(retry_count, " sleep mat_data", mat_data_resp.headers['Retry-After'])
                retry_count += 1
                time.sleep(int(mat_data_resp.headers['Retry-After']) + 5)
                mat_data_resp = requests.get(mat_url, headers=request_header)

            if mat_data_resp.status_code == 200:
                if mat_data_resp.json()['info']['gameMode'] == 'CLASSIC' \
                    and len(mat_data_resp.json()['info']['teams'][0]['bans']) > 0 \
                    and mat_data_resp.json()['info']['participants'][0]['gameEndedInEarlySurrender'] == False \
                    and mat_data_resp.json()['info']['gameDuration'] > 1500 \
                    and mat_data_resp.json()['info']['participants'][find_partiId(mat_data_resp, puuid)]['teamPosition'] == 'BOTTOM':
                    # 소환사 협곡
                    # 밴이 있음
                    # 재시작 경기 x
                    # 15분 이상
                    # 해당 소환사가 바텀라인

                    match_ids.append(mat_id)

            else:
                print('\n\terror ', mat_data_resp.status_code)
                print('\t', puuid)
                break

        start += count

    else:
        print('\n\terror ', match_list_resp.status_code)
        print('\t', match_ids)
        break


0  sleep match_list 90
1  sleep match_list 95


In [ ]:
display(df_puuid2)

In [11]:
df_puuid

,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,KR_6613941315,KR_6613987729,KR_6614204138,KR_6619754667,KR_6621536984,KR_6621619421,KR_6622454112,KR_6623973606,KR_6624290079,KR_6624480871
1,KR_6661857315,KR_6662658092,KR_6662878759,KR_6663816089,KR_6663918544,KR_6663968631,KR_6664044375,KR_6666840236,KR_6667098846,KR_6667275166
2,KR_6649317059,KR_6649399560,KR_6651641733,KR_6656551553,KR_6657626230,KR_6658072186,KR_6658114347,KR_6659075859,KR_6659283267,KR_6659923271
3,KR_6505755344,KR_6506836799,KR_6507063069,KR_6508479817,KR_6508528779,KR_6510608163,KR_6515250854,KR_6520524185,KR_6595497208,KR_6597466848
4,KR_6600350874,KR_6600518396,KR_6600690424,KR_6603571667,KR_6603662270,KR_6606740799,KR_6607340598,KR_6607603522,KR_6607703499,KR_6607898972
5,KR_6675293991,KR_6677031188,KR_6677223696,KR_6678439789,KR_6678471964,KR_6678726867,KR_6679804541,KR_6680129404,KR_6681420330,KR_6681494325
6,KR_6483406063,KR_6512503768,KR_6544457043,KR_6560860780,KR_6561935466,KR_6562544040,KR_6562587759,KR_6563164290,KR_6566183643,KR_6572437459
7,KR_6543581378,KR_6547253696,KR_6549561761,KR_6566278540,KR_6568801782,KR_6568879247,KR_6570507729,KR_6570599405,KR_6570692131,KR_6570780548
8,KR_6643645803,KR_6644176053,KR_6644273750,KR_6645738229,KR_6645902831,KR_6651306856,KR_6651359462,KR_6651825134,KR_6652007155,KR_6652124381
9,KR_6611167883,KR_6611251483,KR_6611273171,KR_6613147937,KR_6613216264,KR_6615021843,KR_6615053056,KR_6615069604,KR_6617034482,KR_6617062718


In [12]:
df_puuid.to_excel('temp_df_puuid.xlsx', index=False)

In [ ]:
display(df_puuid.head(2))
display(df_puuid.tail(2))

In [ ]:
df2 = pd.concat([df, df_puuid], axis=1)

In [ ]:
df2.to_excel('matchid_data.xlsx', index=False)

In [ ]:
df2 = pd.read_excel('matchid_data.xlsx')

In [ ]:
display(df2.head(2))
display(df2.tail(2))

#### 경기 데이터 전처리

In [ ]:
def make_base_dict(puuid, matchid):
    base_dict = {'puuid': puuid, 'matchid': matchid}
    
    return base_dict

In [ ]:
def response_match(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid
    resp_ma = requests.get(url, headers=request_header)

    return resp_ma

In [ ]:
# participant ID 찾기
### match 에서는 pid-1
### timeline 에서는 'pid'
def find_participantId(resp_ma, puuid):
    return resp_ma.json()['metadata']['participants'].index(puuid) + 1

In [ ]:
def make_data_end_dict(resp_ma, parti_id):
    data_end = resp_ma.json()['info']['participants'][parti_id - 1]

    data_end_dict = {'win': data_end['win'],
                     'kills': data_end['kills'],
                     'deaths': data_end['deaths'],
                     'assists': data_end['assists'],
                     'bountyGold': data_end['challenges']['bountyGold'],
                     'turretKills': data_end['turretKills'],
                     'turretsLost': data_end['turretsLost']
                    }
    
    return data_end_dict

In [ ]:
def response_timelines(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid + '/timeline'
    resp_tl = requests.get(url, headers=request_header)
    
    return resp_tl

In [ ]:
def make_data_15_dict(resp_tl, parti_id):
    data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(parti_id)]

    data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                    'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                    'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                    'level15': data_15['level'],
                    'xp15': data_15['xp'],
                    'minionsKilled15': data_15['minionsKilled'],
                    'totalGold15': data_15['totalGold']
                    }
    
    return data_15_dict

In [ ]:
def make_data_15_kda_dict(resp_tl, parti_id):
    data_15_kda_dict = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

    for times in range(0, 16):
        events = resp_tl.json()['info']['frames'][times]['events']

        for i in range(len(events)):
            if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
                
                if parti_id == events[i]['killerId']:
                    data_15_kda_dict['kills15'] += 1

                if parti_id == events[i]['victimId']:
                    data_15_kda_dict['deaths15'] += 1

                if parti_id in events[i]['assistingParticipantIds']:
                    data_15_kda_dict['assists15'] += 1

    return data_15_kda_dict

In [ ]:
match_cols = ['puuid', 'matchid', \
              'win', 'kills', 'deaths', 'assists', 'bountyGold', 'turretKills', 'turretsLost', \
              'totalDamageDone15', 'totalDamageDoneToChampions15', 'totalDamageTaken15', 'level15', 'xp15', 'minionsKilled15', 'totalGold15', \
              'kills15', 'deaths15', 'assists15']

In [ ]:
df_match = pd.DataFrame(columns=match_cols)
df_match

In [ ]:
for i in range(len(df2)):
    puuid = list(df2.loc[[i]]['puuid'])[0]
    match_ids = list(df2.loc[i])[3:]
    
    for matchid in match_ids:
        base_dict = make_base_dict(puuid, matchid)
        
        resp_ma = response_match(matchid)
        if resp_ma.status_code == 429: # Rate limit exceed
            print("sleep(ma)", resp_ma.headers['Retry-After'])
            time.sleep(int(resp_ma.headers['Retry-After']) + 5)
            resp_ma = response_match(matchid)

        parti_id = find_participantId(resp_ma, puuid)
        data_end_dict = make_data_end_dict(resp_ma, parti_id)

        resp_tl = response_timelines(matchid)
        if resp_tl.status_code == 429: # Rate limit exceed
            print("sleep(tl)", resp_tl.headers['Retry-After'])
            time.sleep(int(resp_tl.headers['Retry-After']) + 5)
            resp_tl = response_match(matchid)

        data_15_dict = make_data_15_dict(resp_tl, parti_id)
        data_15_kda_dict = make_data_15_kda_dict(resp_tl, parti_id)

        base_dict.update(data_end_dict)
        base_dict.update(data_15_dict)
        base_dict.update(data_15_kda_dict)
        df_match = df_match.append(base_dict, ignore_index=True)

    time.sleep(5)

In [ ]:
display(df_match.head(2))
display(df_match.tail(2))

In [ ]:
df_match.to_excel('match_log_data.xlsx', index=False)

In [ ]:
df_match = pd.read_excel('match_log_data.xlsx')

#### 연습구간

In [ ]:
# parti_ID 찾기
def find_participantId(response, puuid):
    return response.json()['metadata']['participants'].index(puuid) + 1

In [ ]:
find_participantId(resp_tl, df2['puuid'][0])

In [ ]:
resp_tl.json()['info']['frames'][15]['participantFrames']['9']

In [ ]:
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDone']) # 15분까지 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDoneToChampions']) # 15분까지 챔피언에게 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageTaken']) # 15분까지 받은 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['level']) # 15분 레벨
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['minionsKilled']) # 15분 미니언 처치 수
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['totalGold']) # 15분 골드 누적 획득량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['xp']) # 15분 경험치 누적 획득량

# 게임 종료
print(response.json()['info']['participants'][8]['challenges']['bountyGold']) # 현상금
print(response.json()['info']['participants'][8]['kills']) # 킬
print(response.json()['info']['participants'][8]['deaths']) # 데스
print(response.json()['info']['participants'][8]['assists']) # 어시스트
print(response.json()['info']['participants'][8]['turretKills']) # 자신이 파괴한 포탑 수
print(response.json()['info']['participants'][8]['turretsLost']) # 전체 파괴된 포탑 수
print(response.json()['info']['participants'][8]['win']) # 승패 여부

In [ ]:
resp_tl.json()['info']['frames'][8]['events'][22].keys()

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

##### 테스트

In [ ]:
pud = df2['puuid'][0]
mid = df2['match1'][0]
basedict = {'puuid': pud, 'matchid': mid}

In [ ]:
url1 = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid
resp_end = requests.get(url1, headers=request_header)
part_id = find_participantId(resp_end, pud)

In [ ]:
part_id

In [ ]:
data_end = resp_end.json()['info']['participants'][part_id-1]
data_end_dict = {'win': data_end['win'],
                 'kills': data_end['kills'],
                 'deaths': data_end['deaths'],
                 'assists': data_end['assists'],
                 'bountyGold': data_end['challenges']['bountyGold'],
                 'turretKills': data_end['turretKills'],
                 'turretsLost': data_end['turretsLost']
                 }

In [ ]:
data_end_dict

In [ ]:
url_tl = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid + '/timeline'
resp_tl = requests.get(url_tl, headers=request_header)

In [ ]:
data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(part_id)]
data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                'level15': data_15['level'],
                'xp15': data_15['xp'],
                'minionsKilled15': data_15['minionsKilled'],
                'totalGold15': data_15['totalGold']
                }

In [ ]:
data_15_dict

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

In [ ]:
data_15_kda = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

In [ ]:
for times in range(0, 16):
    events = resp_tl.json()['info']['frames'][times]['events']

    for i in range(len(events)):
        if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
            
            if part_id == events[i]['killerId']:
                data_15_kda['kills15'] += 1

            if part_id == events[i]['victimId']:
                data_15_kda['deaths15'] += 1

            if part_id in events[i]['assistingParticipantIds']:
                data_15_kda['assists15'] += 1

In [ ]:
data_15_kda

In [ ]:
basedict.update(data_end_dict)
basedict.update(data_15_dict)
basedict.update(data_15_kda)

In [ ]:
basedict

In [ ]:
df_match.append(basedict, ignore_index=True)

##### -----

In [ ]:
tempdf = df_match.copy()
tempdf


In [ ]:
temp_dict = {'win':True, 'kills':10, 'deaths':20, 'assists':30}
td2 = {'new': False}

In [ ]:
tempdf = pd.DataFrame(columns=['win', 'kills', 'deaths', 'assists', 'new'])

In [ ]:
for key, value in temp_dict.items():
    print(key, value)
    tempdf.append(temp_dict, ignore_index=True)

tempdf

In [ ]:
temp_dict.update(td2)

In [ ]:
temp_dict

In [ ]:
tempdf.append(temp_dict, ignore_index=True)

In [ ]:
requests.get('http://ddragon.leagueoflegends.com/cdn/13.17.1/data/en_US/item.json').json()['data']['1001']['image']['full']

In [ ]:
url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + df2['match1'][0] + '/timeline'
response = requests.get(url, headers=request_header)
printjs(response.json())

In [ ]:
response.json()['info'].keys()

In [ ]:
response.json()['info']['frames']

In [ ]:
temp = response.json()['info']
len(temp['frames'])

In [ ]:
for x in temp['frames']:
    del x['participantFrames']

In [ ]:
temp['frames']

#### 15분
* 서렌 가능 시간
* 포탑 방패가 사라지는 14분 + 이후 타워가 깨질 수 있는 시간 약 1분
* 챔피언 성능이 달라지는 1코어 이상(1.5코어) 아이템 소지

#### 10분?
* 다른 라이너의 큰 개입 없이 라인전이 이루어지는 시간
* 서포터까지도 충분히 궁극기를 배우고 교전에 적극 참여하기 충분한 시간
* 12~13분이면 라인전이 끝남, 15분은 거의 3번째 용